In [4]:
import pandas as pd


data = pd.read_parquet(r"data\parquet\final.parquet")
parties = ["AfD", "DIE_GRÜNEN", "DIE_LINKE", "FDP", "SPD", "Union"]



In [22]:
all_data = None

for party in parties:
    party_data = data[[
        "vote", "drucksache_id", "beschlussempfehlung", "type", "summary", "date", "category",
        f"{party}_prediction", f"{party}_ground_truth", 
    ]].rename(
        columns={
            f"{party}_prediction": "prediction",
            f"{party}_ground_truth": "ground_truth",
        }
    )
    party_data = party_data[party_data["ground_truth"].notna()]
    party_data["party"] = party
    if all_data is None:
        all_data = party_data
    else:
        all_data = pd.concat([all_data, party_data], ignore_index=True)

all_data["partyline_correct"] = all_data["prediction"] == all_data["ground_truth"]

In [40]:
filtered = all_data.groupby(["category"]).filter(lambda x: len(x) >= 20)
result = filtered.groupby(["category"])["partyline_correct"].mean()
result.sort_values(ascending=False, inplace=True)
result

category
Entwicklungszusammenarbeit - Entwicklungsprojekte, humanitäre Hilfe, internationale Zusammenarbeit                0.887640
Verteidigung & Sicherheit - Militär, Verteidigungsstrategie, Bundeswehr, Rüstung                                  0.827131
Außenpolitik & Europäische Angelegenheiten - Diplomatie, internationale Beziehungen, EU-Politik                   0.775000
Wirtschaft & Energie - Industriepolitik, Mittelstand, Energieversorgung, Wirtschaftsordnungen                     0.647541
Inneres & Migration - Innere Sicherheit, öffentliche Verwaltung, Migration, Staatsbürgerschaft                    0.627168
Umwelt, Klima & Naturschutz - Umweltschutz, Klimapläne, Artenschutz, nukleare Sicherheit                          0.625954
Finanzen - Steuern, Staatsbudget, Haushalts- und Finanzpolitik                                                    0.605634
Wohnen & Stadtentwicklung - Wohnungsbau, Städtebau, Bauordnung, Städtebauförderung                                0.588235
Verkehr

In [68]:
all_data["category"].unique()

array(['Verteidigung & Sicherheit - Militär, Verteidigungsstrategie, Bundeswehr, Rüstung',
       'Inneres & Migration - Innere Sicherheit, öffentliche Verwaltung, Migration, Staatsbürgerschaft',
       'Finanzen - Steuern, Staatsbudget, Haushalts- und Finanzpolitik',
       'Umwelt, Klima & Naturschutz - Umweltschutz, Klimapläne, Artenschutz, nukleare Sicherheit',
       'Wohnen & Stadtentwicklung - Wohnungsbau, Städtebau, Bauordnung, Städtebauförderung',
       'Gesundheit - Gesundheitssystem, Krankenversicherung, Arzneimittelregulierung, Pandemie- und Präventionspolitik',
       'Entwicklungszusammenarbeit - Entwicklungsprojekte, humanitäre Hilfe, internationale Zusammenarbeit',
       'Wirtschaft & Energie - Industriepolitik, Mittelstand, Energieversorgung, Wirtschaftsordnungen',
       'Landwirtschaft & Ernährung - Agrarpolitik, Ernährungssicherheit, Ländliche Entwicklung',
       'Justiz & Verbraucherschutz - Rechtsprechung, Gesetzgebung, Verbraucherschutz, Datenschutz',
       '

In [1]:
from src.utils.openai_utils import classify_lobbyregister_entry
import pandas as pd
with open(r"data/tmp/lobbyregister.json", "r") as f:
    import json
    data = json.load(f)
data = pd.DataFrame(data["results"])

In [2]:
classify_lobbyregister_entry(data["activitiesAndInterests"].iloc[0])

LobbyregisterEntryDistribution(categories=[CategoryEntry(category='Entwicklungszusammenarbeit - Entwicklungsprojekte, humanitäre Hilfe, internationale Zusammenarbeit', percentage=50), CategoryEntry(category='Gesundheit - Gesundheitssystem, Krankenversicherung, Arzneimittelregulierung, Pandemie- und Präventionspolitik', percentage=25), CategoryEntry(category='Arbeit & Soziales - Arbeitsmarktpolitik, Sozialversicherung, Renten, Integration', percentage=10), CategoryEntry(category='Bildung, Familie & Jugend - Schulen, Hochschulen, Familienförderung, Kinder- und Jugendpolitik', percentage=10), CategoryEntry(category='Inneres & Migration - Innere Sicherheit, öffentliche Verwaltung, Migration, Staatsbürgerschaft', percentage=5)])

In [4]:
data["activitiesAndInterests"].iloc[0]

{'activity': {'code': 'ACT_NONPROFIT_ORGA_V2',
  'de': 'Nichtregierungsorganisation (NGO)',
  'en': 'Non-governmental organization (NGO)'},
 'typesOfExercisingLobbyWork': [{'code': 'SELF_OPERATED_OWN_INTEREST',
   'de': 'Die Interessenvertretung wird in eigenem Interesse selbst wahrgenommen',
   'en': 'Interest representation is self-performed in its own interest'}],
 'fieldsOfInterest': [{'code': 'FOI_OTHER',
   'de': 'Sonstige Interessenbereiche',
   'en': 'Other areas',
   'fieldOfInterestText': 'humanitäre Hilfe, Katastrophenvorsorge, globale Gesundheit, gemeindebasierte Entwicklung, Inklusion'},
  {'code': 'FOI_FA_INTERNATIONAL',
   'de': 'Internationale Beziehungen',
   'en': 'International relations'},
  {'code': 'FOI_REFUGEE_INTEGRATION_POLICY',
   'de': 'Migration, Flüchtlingspolitik und Integration',
   'en': 'Migration, refugee policy and integration'},
  {'code': 'FOI_HEALTH_OTHER',
   'de': 'Sonstiges im Bereich "Gesundheit"',
   'en': 'Other in the field of "Health"'},
  